# Parsing through JSON files to obtain dataframe with mutations found on Nextstrain's visualization of H7N9 IAV

Author: Luis Antonio Haddock III & Katarina M Braun, MD/PhD 

Adapted from: Louise Moncla, PhD (https://journals.plos.org/plospathogens/article?id=10.1371/journal.ppat.1008191)


Last edit: 2022-03-27




In [1]:
import sys, subprocess, glob, os, shutil, re, importlib, csv, json
from subprocess import call
import collections
from collections import Counter
from Bio import SeqIO
from Bio import Seq
import Bio.Phylo
import pandas as pd
import numpy as np
import rpy2
from scipy import stats
%reload_ext rpy2.ipython
from io import BytesIO as csio
import requests
import matplotlib.pyplot as plt
import matplotlib
from functools import reduce

# we might not use all modules in this analysis

In [2]:
segments = ['ha', 'pa', 'na', 'pb1', 'pb2', 'mp', 'ns', 'np']


for file in segments:
    links = ['http://data.nextstrain.org/flu_avian_h7n9_' + file + '.json']
    
    for link in links:
        name = link.split("/")[3].lower()
        path = ['JSON_files/' + name]
        tree_json=json.load(csio(requests.get(link).content))
        tree_json_2 = tree_json['tree']
        out_file = open(name, "w")
        json.dump(tree_json_2, out_file, indent = 2) 
        out_file.close() 
        

In [3]:
%%bash
# creating a directory to store all json files (internal use only)

rm -rf JSON_files/
mkdir -p JSON_files/

mv *.json JSON_files/

In [4]:
# function to read json file and store it in a variable 

def read_json(file_name):
    try:
        handle = open(file_name, 'r')
    except IOError:
        pass
    else:
        data = json.load(handle)
        handle.close()
    return data

In [5]:
# reading the tree json file in V2 (different code is used for old V1 JSON format)

def json_to_tree(json_dict, root=True):
    # Check for v2 JSON which has combined metadata and tree data.
    
    if root and "meta" in json_dict and "tree" in json_dict:
        json_dict = json_dict["tree"]
    
    node = Bio.Phylo.Newick.Clade()
    
    if "name" in json_dict:
        node.name = json_dict["name"] #strain was changed to name from v1 to v2
    else:
        node.name = json_dict["strain"]

    if "children" in json_dict:
        # Recursively add children to the current node.
        node.clades = [json_to_tree(child, root=False) for child in json_dict["children"]]

    # Assign all non-children attributes.
    for attr, value in json_dict.items():
        if attr != "children":
            setattr(node, attr, value)

   # Only v1 JSONs support a single `attr` attribute.
    if hasattr(node, "attr"):
        node.numdate = node.attr.get("num_date")
        node.branch_length = node.attr.get("div")

        if "translations" in node.attr:
            node.translations = node.attr["translations"]
    elif hasattr(node, "node_attrs"):
        node.branch_length = node.node_attrs.get("div")

    if root:
        node = annotate_parents(node)

        
    return node

In [6]:
# Annotations 

def annotate_parents(tree):
    # Get all parent nodes by node.
    parents_by_node = all_parents(tree)
    #print("number of parent nodes is: ", len(parents_by_node))

    # Next, annotate each node with its parent.
    for node in tree.find_clades():
        if node == tree.root:
            node.up = None
        else:
            node.up = parents_by_node[node]

    # Return the tree.
    return tree

def all_parents(tree):
    parents = {}
    for clade in tree.find_clades(order='level'):
        for child in clade:
            parents[child] = clade
    return parents

In [7]:
# Parsing thorugh tree in V2 (adapted from Louise Moncla, PhD)

def parse_tree_and_return_muts(tree, gene): 
    

#lists to store aa mutations
    avian_mutations = []
    human_mutations = []
    
    for clade in tree.find_clades(): 
        tips_list = []
        hosts_list = []
        host_values = []
        
        for terminal in clade.get_terminals():
            tips_list.append(terminal.name)
            hosts_list.append(terminal.node_attrs['host'])
            
        for a_dict in hosts_list:
            host_values.append(a_dict['value'])
            
            if set(host_values) == {"Human"}:
                
                up1_hosts_list = []
                up1_temp_host = []
                up1_tips_list = []
                
                if clade.up == None:
                    print("root")
                
                else:
                    for x in clade.up.get_terminals():
                        up1_tips_list.append(x.name)
                        up1_temp_host.append(x.node_attrs['host'])
                    

                    for b_dict in up1_temp_host:
                        up1_hosts_list.append(b_dict['value'])

                    #we look through nodes for human hosts and extract the 
                    #mutations that leads to said branch
                    if set(up1_hosts_list) == {'Human'} and 'mutations' in clade.branch_attrs and 'mutations' in clade.branch_attrs != {}:  
                        if gene in clade.branch_attrs['mutations']:

                            aa_muts = clade.branch_attrs['mutations']
                            
                            for mut in aa_muts[gene]:

                                human_mutations.append(mut)
        
                            
            
            
        for a_dict in hosts_list:
            host_values.append(a_dict['value'])
            
            if set(host_values) == {"Avian"}:
                
                up2_hosts_list = []
                up2_temp_host = []
                up2_tips_list = []
                
                #for x in clade.get_terminals(): #if I fo this then it works with HA
                """add same thing down here"""
                if clade.up == None: 
                    print("root")
                else:
                    for x in clade.up.get_terminals():
                        up2_tips_list.append(x.name)
                        up2_temp_host.append(x.node_attrs['host'])

                    for b_dict in up2_temp_host:
                        up2_hosts_list.append(b_dict['value'])

                    # if the node above contains avan samples, then clade should be set to avian
                    if set(up2_hosts_list) == {'Avian'} and 'mutations' in clade.branch_attrs and 'mutations' in clade.branch_attrs != {}:  
                        if gene in clade.branch_attrs['mutations']:

                            aa_muts = clade.branch_attrs['mutations']
                            for mut in aa_muts[gene]:

                                avian_mutations.append(mut)

    
    return(avian_mutations, human_mutations)

# Counting the number of times each mutation occurrs in each list



In [8]:
# For simplicity we change aa code format

aa_dict = {"A":"Ala","R":"Arg","N":"Asn","D":"Asp","C":"Cys","Q":"Gln","E":"Glu","G":"Gly","H":"His",
           "I":"Ile","L":"Leu","K":"Lys","M":"Met","F":"Phe","P":"Pro","S":"Ser","T":"Thr","W":"Trp",
           "Y":"Tyr","V":"Val", "*":"stop"}


In [9]:
def convert_aa_changes(bb, hh):
    bb2 = []
    #bh2 = []
    hh2 = []
    
    for a in bb:
        # convert aa symbols to 3-letter abbreviations
        first_aa = a[0]
        new_first = aa_dict[first_aa]
        last_aa = a[-1]
        new_last = aa_dict[last_aa]
        a = a.replace(first_aa, new_first)
        a = a.replace(last_aa, new_last)
        bb2.append(a)  
#    for a in bh:
#        # convert aa symbols to 3-letter abbreviations
#        first_aa = a[0]
#        new_first = aa_dict[first_aa]
#        last_aa = a[-1]
#        new_last = aa_dict[last_aa]
#        a = a.replace(first_aa, new_first)
#        a = a.replace(last_aa, new_last)
#        bh2.append(a)
    for a in hh:
        # convert aa symbols to 3-letter abbreviations
        first_aa = a[0]
        new_first = aa_dict[first_aa]
        last_aa = a[-1]
        new_last = aa_dict[last_aa]
        a = a.replace(first_aa, new_first)
        a = a.replace(last_aa, new_last)
        hh2.append(a)
        
    return(bb2, bh2, hh2)

In [10]:
def return_SNP_counts(bb, hh):
    
    # use Counter to count the number of times each amino acid change is detected in each list; print total unique SNPs
    #bird_to_bird_count = Counter(bird_to_bird)
    bird = Counter(avian_mutations)
    human = Counter(human_mutations)
    
    # get a complete list of all of the SNPs identified
    all_SNPs = set(avian_mutations + human_mutations)
    
    # loop through and count how many times each SNP occurs in each dataset 
    all_counts = {}

    for a in all_SNPs:
                
        if a in bird:
            bird_snp = bird[a]
        else:
            bird_snp = 0

        if a in human:
            human_snp = human[a]   
        else:
            human_snp = 0


        all_counts[a] = {"bird_snp":bird_snp, "human_snp": human_snp}
    return(all_counts)

In [16]:
def generate_dataframe(all_counts, gene, df):
    df1 = pd.DataFrame.from_dict(all_counts, orient='index')
    df1 = df1.reset_index()
    df1.columns=['coding_region_change','avian_mutations','human_mutations'] #, 'avian_strains', 'human_strains']
    df1['gene'] = gene
    df = pd.concat([df,df1])
    return(df)

# Running functions across JSON files

In [17]:
#old_code
df = pd.DataFrame()

for f in glob.glob("JSON_files/flu_avian_h7n9_*.json"):
    tree = read_json(f)
    tree = json_to_tree(tree)
    gene = f.split("_")[3].upper()
    gene = gene.split(".")[0].upper()

    
    if gene == "MP":
        genes = ["M1","M2"]
        for g in genes: 
            avian_mutations, human_mutations  = parse_tree_and_return_muts(tree, g)
#            #bird_to_bird, bird_to_human, human_to_human = convert_aa_changes(bird_to_bird, bird_to_human, human_to_human)
            all_counts = return_SNP_counts(avian_mutations, human_mutations )
            df = generate_dataframe(all_counts, g, df)
    
    elif gene == "NS":
        genes = ["NS1","NS2"]
        for g in genes: 
            avian_mutations, human_mutations  = parse_tree_and_return_muts(tree, g)
            #bird_to_bird, bird_to_human, human_to_human = convert_aa_changes(bird_to_bird, bird_to_human, human_to_human)
            all_counts = return_SNP_counts(avian_mutations, human_mutations )
            df = generate_dataframe(all_counts, g, df)
    elif gene != "MP" and gene != "NS": # and gene != 'HA': 
        # run everything
        avian_mutations, human_mutations = parse_tree_and_return_muts(tree, gene)
        #bird_to_bird, bird_to_human, human_to_human = convert_aa_changes(bird_to_bird, bird_to_human, human_to_human)
        all_counts = return_SNP_counts(avian_mutations, human_mutations)
        df = generate_dataframe(all_counts, gene, df)
        

root
root
root
root
root


ValueError: Length mismatch: Expected axis has 1 elements, new values have 3 elements

In [15]:
#df.to_csv('mutations_with_strains.tsv', sep='\t')

df['gene'] = df['gene'].str.replace('NS1', 'NS')
df['gene'] = df['gene'].str.replace('NS2', 'NS')
df['gene'] = df['gene'].str.replace('M1', 'MP')
df['gene'] = df['gene'].str.replace('M2', 'MP')
df = df.drop_duplicates()
df['gene'].unique()

array([], dtype=object)

In [12]:
df['region_gene'] = df['coding_region_change'] + "_" + df['gene']
df['site'] = df['gene'] + " " + df['coding_region_change']
df.to_csv('full_nextstrain_mutations_table.tsv', sep='\t')
df


,coding_region_change,avian_mutations,human_mutations,gene,region_gene,site
0,Q400K,0,1,HA,Q400K_HA,HA Q400K
1,L517I,8,0,HA,L517I_HA,HA L517I
2,A169T,2,7,HA,A169T_HA,HA A169T
3,M547I,0,1,HA,M547I_HA,HA M547I
4,D283A,0,5,HA,D283A_HA,HA D283A
...,...,...,...,...,...,...
215,L136M,0,1,NP,L136M_NP,NP L136M
216,F420Y,0,1,NP,F420Y_NP,NP F420Y
217,T215A,0,1,NP,T215A_NP,NP T215A
218,T442A,1,0,NP,T442A_NP,NP T442A


In [94]:
print('Success, your code works!!! 🙏🏻')

Success, your code works!!! 🙏🏻
